In [1]:
from keras.layers import Input, Dense, Lambda, Flatten, Activation, Merge, Concatenate, Add
from keras import layers
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.objectives import binary_crossentropy
from keras.callbacks import LearningRateScheduler

import numpy as np
from scipy.stats import sem

from keras import backend as K
from keras.models import load_model
import tensorflow as tf

from models import vgg
from models.cvae import CVAE
from utils.angles import deg2bit, bit2deg
from utils.losses import mad_loss_tf, cosine_loss_tf, von_mises_loss_tf, maad_from_deg
from utils.losses import gaussian_kl_divergence_tf, gaussian_kl_divergence_np
from utils.losses  import von_mises_log_likelihood_tf, von_mises_log_likelihood_np
from utils.towncentre import load_towncentre
from utils.experiements import get_experiment_id

Using TensorFlow backend.


In [2]:
xtr, ytr_deg, xval, yval_deg, xte, yte_deg = load_towncentre('data/TownCentre.pkl.gz', canonical_split=True)
image_height, image_width = xtr.shape[1], xtr.shape[2]
ytr_bit = deg2bit(ytr_deg)
yval_bit = deg2bit(yval_deg)
yte_bit = deg2bit(yte_deg)

image_height, image_width, n_channels = xtr.shape[1:]
flatten_x_shape = xtr[0].flatten().shape[0]
phi_shape = yte_bit.shape[1]

In [7]:
vgg_fixed_kappa = vgg.BiternionVGG(image_height=image_height,
                                   image_width=image_width,
                                   n_channels=3,
                                   predict_kappa=False,
                                   fixed_kappa_value=3.0)

vgg_fixed_kappa.model.load_weights('/Users/sergey/BiternionNet/logs/vggbit_likelihood_learned_kappa_Adadelta_200epochs_batch10_a87f25f2a6b61f752c35/vgg_bit_vm_likelihood_town.best.weights.h5')

In [9]:
def eval_bvgg_model(best_vgg, x, ytrue_deg, ytrue_bit, data_part):

    ypreds = best_vgg.model.predict(x)
    ypreds_bit = ypreds[:, 0:2]
    ypreds_deg = bit2deg(ypreds_bit)

    if best_vgg.predict_kappa:
        kappa_preds = ypreds[:, 2:]
    else:
        kappa_preds = np.ones([ytrue_deg.shape[0], 1]) * best_vgg.fixed_kappa_value

    loss = maad_from_deg(ypreds_deg, ytrue_deg)
    
    results = dict()
    
    results['mean_loss_'+data_part] = float(np.mean(loss))
    results['std_loss_'+data_part] = float(np.std(loss))
    print("MAAD error (%s) : %f ± %f" % (data_part,
                                         results['mean_loss_'+data_part],
                                         results['std_loss_'+data_part]))

    results['mean_kappa_'+data_part] = float(np.mean(kappa_preds))
    results['std_kappa_'+data_part] = float(np.std(kappa_preds))
    print("predicted kappa (%s) : %f ± %f" % (data_part,
                                                  results['mean_kappa_'+data_part],
                                                  results['std_kappa_'+data_part]))

    log_likelihoods = von_mises_log_likelihood_np(ytrue_bit, ypreds_bit, kappa_preds,
                                                  input_type='biternion')
    results['log_likelihood_mean_'+data_part] = float(np.mean(log_likelihoods))
    results['log_likelihood_sem_'+data_part] = float(sem(log_likelihoods, axis=None))
    print("log-likelihood (%s) : %f ± %fSEM" % (data_part,
                                                results['log_likelihood_mean_'+data_part],
                                                results['log_likelihood_sem_'+data_part]))
    
    return results

In [11]:
val_results = eval_bvgg_model(vgg_fixed_kappa, xval, yval_deg, yval_bit, 'validation')

MAAD error (validation) : 47.180345 ± 45.412062
predicted kappa (validation) : 3.000000 ± 0.000000
log-likelihood (validation) : -1.711596 ± 0.061680SEM


In [9]:
_eval_vgg_model(vgg_fixed_kappa, xtr, ytr_deg, ytr_bit, 'train')

NameError: name 'predict_kappa' is not defined

#### CVAE model 

In [12]:
n_u = 8
cvae_best = CVAE(n_hidden_units=n_u)
cvae_ckpt_path = 'logs/cvae.full_model.weights.hdf5'
cvae_best.full_model.load_weights(cvae_ckpt_path)

In [13]:
def eval_cvae_model(cvae_model, x, ytrue_deg, ytrue_bit, data_part):
    
    n_samples = x.shape[0]

    cvae_preds = cvae_model.full_model.predict([x, ytrue_bit])
    elbo_te, ll_te, kl_te = cvae_model._cvae_elbo_loss_np(ytrue_bit, cvae_preds)

    ypreds = cvae_model.decoder_model.predict(x)
    ypreds_bit = ypreds[:,0:2]
    kappa_preds_te = ypreds[:,2:]

    ypreds_deg = bit2deg(ypreds_bit)

    loss_te = maad_from_deg(ytrue_deg, ypreds_deg)
    mean_loss_te = np.mean(loss_te)
    std_loss_te = np.std(loss_te)

    print("MAAD error (%s) : %f ± %f" % (data_part, mean_loss_te, std_loss_te))

    print("kappa (%s) : %f ± %f" % (data_part, np.mean(kappa_preds_te), np.std(kappa_preds_te)))

    log_likelihood_loss = von_mises_log_likelihood_np(ytrue_bit, ypreds_bit, kappa_preds_te,
                                                         input_type='biternion')

    print("ELBO (%s) : %f ± %f SEM" % (data_part, np.mean(-elbo_te), sem(-elbo_te)))

    print("KL(encoder|prior) (%s) : %f ± %f SEM" % (data_part, np.mean(-kl_te), sem(-kl_te)))

    print("log-likelihood (%s) : %f±%fSEM" % (data_part, 
                                              np.mean(log_likelihood_loss), 
                                              sem(log_likelihood_loss)))
    return

In [14]:
eval_cvae_model(cvae_best, xte, yte_deg, yte_bit, 'train')
eval_cvae_model(cvae_best, xval, yval_deg, yval_bit, 'validation')
eval_cvae_model(cvae_best, xte, yte_deg, yte_bit, 'test')

MAAD error (train) : 24.149429 ± 31.282352
kappa (train) : 12.268851 ± 10.783769
ELBO (train) : -0.694701 ± 0.054878 SEM
KL(encoder|prior) (train) : -0.000583 ± 0.000055 SEM
log-likelihood (train) : -0.690100±0.054472SEM
MAAD error (validation) : 23.977060 ± 32.969274
kappa (validation) : 12.130845 ± 10.135998
ELBO (validation) : -0.650484 ± 0.058520 SEM
KL(encoder|prior) (validation) : -0.000704 ± 0.000079 SEM
log-likelihood (validation) : -0.650794±0.058297SEM
MAAD error (test) : 24.041479 ± 31.067524
kappa (test) : 12.259947 ± 10.808006
ELBO (test) : -0.692449 ± 0.054575 SEM
KL(encoder|prior) (test) : -0.000583 ± 0.000055 SEM
log-likelihood (test) : -0.682615±0.052969SEM
